# Lab06 - Intrusion Detection

## General Information

**Assistents**
- Jamila Alsayed Kassem
- Leonardo Boldrini
- Wouter Petri
- Frederick Kreuk
- Kyrian Maat
- Robin Slot
- Emiel Wiedijk
- Robert van Straten
- Ralph Erkamps

**Lab dates**  
13 or 14 October and 18 or 19 October 2022

**Deadline**  
20 October 2022 at 12:00 CEST

**Total points**  
20

## Lab Description

In this lab you will receive several PCAP files. These files will contain network-based attacks. Your task in this assignment is to write code to detect these attacks and classify the type thereof. The attacks are all different types of DDoS attacks. Your perspective is that of the network engineer, trying to protect their network against these DDoS attacks. You want to classify attacks so that you can create filters to omit malevolent traffic.

For the submission of this lab, **you will only have to hand in this Jupyter Notebook**, which should contain all your code and explanations as indicated in the Notebook. Please fill in any missing sections of code, and provide explanations where these are asked.

**This lab must be done in pairs. Before you submit, you have to register into a group on CodeGrade.**

## Setup

The following few blocks contain general imports and functions that you may use during this lab. Of note is the library ScaPy, which provides a PCAP file reader for use during this lab. 

In [1]:
from scapy.all import PcapReader
from scapy.plist import PacketList
from typing import Generator

C:\Users\Filip\AppData\Local\Programs\Python\Python38\lib\site-packages\scapy\layers\ipsec.py:471: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\Filip\AppData\Local\Programs\Python\Python38\lib\site-packages\scapy\layers\ipsec.py:485: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


### Pcap Reader

The code below contains the simple PCAP Reader as provided by ScaPy. It reads from a PCAP file, providing either all packets if the window size is set to 0 or lower, or otherwise only a small window of packets.

In [2]:
def pcap_reader(fpath: str, window_size: int = 50) -> Generator[PacketList, None, None]:
    """PCAP reader generator function.

    Parameters:
    -----------    
    fpath (str): path to pcap file
    window_size (int): number of packets to read at a time

    Output:
    -------
    If windows_size > 0:
        Generator producing PacketLists
    Else:
        PacketList
    """
    reader = PcapReader(fpath)

    if window_size > 0:
        packets = True
        while packets:
            packets = reader.read_all(count=window_size)
            yield packets
    else:
        yield reader.read_all()

The following code gives an example for how to use the PcapReader, as well as what an individual packet looks like.

In [3]:
# The following code reads the entire PCAP file, and outputs all source addresses of TCP packets:
addresses = set()
for packets in pcap_reader('PCAPs_1/easy.pcap', window_size=-1):
    for packet in packets:
        if 'TCP' in packet:
            addresses.add(packet['IP'].src)
print(f'Source addresses: {addresses}')

Source addresses: {'10.0.0.2', '10.128.0.2'}


In [4]:
# The following code reads a window of 50 PCAP packets at a time, and outputs all destination addresses of TCP packets:
addresses = set()
for packets in pcap_reader('PCAPs_1/easy.pcap', window_size=50):
    for packet in packets:
        if 'TCP' in packet:
            addresses.add(packet['IP'].dst)
print(f'Destination addresses: {addresses}')

Destination addresses: {'10.0.0.2', '10.128.0.2'}


In [5]:
# Finally, this code shows what the final packet of the easy.pcap file looks like:
print(packet.show())

###[ Ethernet ]### 
  dst       = 42:01:0a:f0:00:17
  src       = 42:01:0a:f0:00:01
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 44
     id        = 0
     flags     = DF
     frag      = 0
     ttl       = 55
     proto     = tcp
     chksum    = 0x2f49
     src       = 10.128.0.2
     dst       = 10.0.0.2
     \options   \
###[ TCP ]### 
        sport     = http
        dport     = 4874
        seq       = 361191667
        ack       = 1514175688
        dataofs   = 6
        reserved  = 0
        flags     = SA
        window    = 29200
        chksum    = 0xb8a5
        urgptr    = 0
        options   = [('MSS', 1460)]

None


## Assignment 1

In this assignment, you will analyze entire PCAP files containing different kinds of DDoS attacks. For each attack, analyze the corresponding PCAP file to create a fingerprint of the attack. **You may analyze the PCAP file in full, thus be sure to set the window size to -1.** The fingerprint should contain the type of attack and the victim's IP address. Additionally the source(s) of the attack should be included, print only up to the first 50. Lastly you should include the percentage of traffic that is part of the attack and what percentage of IPs are contributing to the attack. In your malicious traffic percentage calculation only consider traffic **to** the victim IP. For your percentage IPs calculation the Victim's IP should be excluded. **Create a function to generate this fingerprint based on the attack**, containing the following information:  
- Attack type: SYN-FLOOD/DNS-AMP/FRAG  
- Victim IP: X.X.X.X 
- Sources: [IP, IP, ...]  
- Percentage traffic: PERCENTAGE
- Percentage IPs: PERCENTAGE

All PCAP files for this assignment can be found in the folder `PCAPs_1`.  
**In addition to writing the code to detect the different kinds of attacks, we also want you to explain your method of detecting the attack in the markdown cell below the code.**

### Task 1.1 - Detecting Syn Flood Attacks (4 points)

The first PCAP file (easy.pcap) we supply is simple, and will contain a straightforward SYN flood attack. In this attack an attacker keeps sending SYN packages, but never replies to the returned SYN ACK with another ACK. This leaves connections in a sort of limbo. Think about what the unique characteristics and patterns are of such a SYN flood attack, and write code to detect this.

For more information on SYN flood attacks, see chapter 3, page 254 of the book.

In [6]:
# Your code here

#### Give an explanation of your method here...

### Task 1.2 - Detecting DNS Amplification Attacks (4 points)

The second PCAP file (medium.pcap) contains a DNS based amplification attack. This is the more advanced part of this assignment, so be creative in what makes a DNS amplification attack unique, and how you could detect this. Then, write code to detect this attack.

For more information on DNS amplification attacks, see here: https://www.cloudflare.com/learning/ddos/dns-amplification-ddos-attack/

In [7]:
# Your code here

### Give an explanation of your method here...

### Task 1.3 - Detecting Packet Fragmentation Attacks (4 points)

Another attack vector is exploiting fragmentation. The last PCAP file (hard.pcap) contains an attack that exploits fragmentation of packets. Think about how fragmentation can be used to launch a DDoS attack, and how you can detect it in code. Write that code.

For more information on packet fragmentation attacks, see here: https://www.imperva.com/learn/ddos/ip-fragmentation-attack-teardrop/

In [36]:
# Your code here
# attacking_ip = set()
# packet_dict = dict()
# number_of_packets = 0
# for packets in pcap_reader('PCAPs_1/hard.pcap', window_size=-1):
#     for packet in packets:
#         if 'IP' in packet and not 'DNS' in packet:
#             if (packet['IP'].id, packet['IP'].src, packet['IP'].dst) not in packet_dict:
#                 packet_dict[(packet['IP'].id, packet['IP'].src, packet['IP'].dst)] = []
#             packet_dict[(packet['IP'].id, packet['IP'].src, packet['IP'].dst)].append((packet['IP'].frag, packet['IP'].flags, packet['IP'].len))
#
# b = 8
# packet_header = 20
# sources = set()
# victims = set()
# attack_type = None
# for packet_id, packet_fragments in packet_dict.items():
#     fragment_missing = False
#     for index, packet_fragment in enumerate(packet_fragments):
#         if packet_fragment[2] > 1486:
#             fragment_missing = True
#             break
#         if index == 0 and packet_fragment[0] != 0:
#             fragment_missing = True
#             break
#         if index > 0 and packet_fragment[0] * b > packet_fragments[index - 1][0] * b + packet_fragments[index - 1][2] - packet_header:
#             fragment_missing = True
#             break
#         if packet_fragment == packet_fragments[-1] and 'MF' == packet_fragment[1]:
#             fragment_missing = True
#             break
#     if fragment_missing:
#         sources.add(packet_id[1])
#         victims.add(packet_id[2])
#         attack_type = "fragmentation attack"
#
# all_traffic = 0
# malicious_traffic = 0
# for packets in pcap_reader('PCAPs_1/hard.pcap', window_size=-1):
#     for packet in packets:
#         proto = None
#         if 'IP' in packet:
#             proto = 'IP'
#         elif 'TCP' in packet:
#             proto = 'TCP'
#         else:
#             proto = 'Ethernet'
#         if packet[proto].dst == list(victims)[0]:
#             all_traffic += 1
#             if packet[proto].src in sources:
#                 malicious_traffic += 1
#
# malicious_ip = 0
# all_ip = 0
# for _, source, destination in packet_dict.keys():
#     if source in sources and destination == list(victims)[0]:
#         malicious_ip += 1
#         all_ip += 1
#     elif destination == list(victims)[0]:
#         all_ip += 1
#
# print(sources)
# print(victims)
# print(f"attack type: {attack_type}")
# print(f"percentage traffic = {malicious_traffic / all_traffic}")
# print(f"percentage ip = {malicious_ip / all_ip}")

{'247.110.43.125', '117.245.186.113', '45.65.205.247', '175.77.153.15', '111.203.36.215', '103.253.55.123', '117.87.171.83', '223.119.139.47', '111.73.155.237', '109.233.124.211', '103.215.74.221', '253.80.119.15', '59.93.135.83', '109.109.75.23', '117.93.109.85', '95.121.219.243', '111.205.182.117', '253.200.183.17', '115.249.86.81', '97.245.56.241', '109.205.104.187', '253.190.37.111', '109.125.15.121', '109.205.104.251', '107.219.180.185', '117.93.217.95', '117.223.190.237', '103.231.12.125', '103.215.190.91', '101.89.139.85', '123.245.101.237', '101.227.40.117', '113.115.191.111', '103.215.72.111', '117.209.140.117', '111.243.218.145', '253.190.121.23', '105.207.59.123', '95.243.251.22', '175.211.74.83', '111.203.89.239', '95.127.39.61', '103.231.56.115', '111.229.139.119', '117.241.168.79', '121.91.203.55', '111.251.9.181', '99.215.106.23', '109.235.142.141', '107.91.89.91', '51.227.20.87', '111.223.202.15', '109.199.106.125', '119.105.173.159', '107.99.47.45', '97.83.155.83', '10

### Give an explanation of your method here...
The idea is that we analyze each packet that is being fragmented.
If it's missing a fragment whether it's the initial one (offset 0)
or the middle one the $n$th fragment's offset is greater than the
$(n-1)$th offset + $(n-1)$th length. Another case is if the last
fragment says there should be more fragments in that packet, but
they were never received by the other side. We can also introduce
a criterion based on the size of the packet fragment. As stated in
the link provided to the task, most network use an MTU of 1500bytes.
Hence, we will also identify the fragments above 1500 as attacks.


## Assignment 2

For this next part of the assignment, you will analyze three larger PCAP files. These PCAP files contain normal traffic, in addition to a possible attack of one of the types from Assignment 1. For this assignment, you will once again need to analyze the files, but you may not look at the file in its entirety. Instead, you may only use a **window size of 50**, to simulate analyzing real time traffic. For each of the files, create a fingerprint where possible as you did in Assignment 1, **but also note when the attack starts, and when it stops**. The format will thus be as follows:  
- Attack type: SYN-FLOOD/DNS-AMP/FRAG  
- Victim IP: X.X.X.X 
- Sources: [IP, IP, ...]  
- Percentage traffic: PERCENTAGE
- Percentage IPs: PERCENTAGE
- Start attack: PACKET_NR
- End attack: PACKET_NR

All PCAP files for this assignment can be found in the folder `PCAPs_2`.   
**Once again, in addition to writing the code to detect the different kinds of attacks, we also want you to explain your method of detecting the attack in the markdown cell below the code.**

### Task 2.1 Large_1 (2 points)

Analyze the file `PCAPs_2/Large_1`.

In [38]:
# Your code here
# b = 8
# packet_header = 20
# sources = set()
# victims = set()
# attack_type = None
# for packet_id, packet_fragments in packet_dict.items():
#     fragment_missing = False
#     for index, packet_fragment in enumerate(packet_fragments):
#         if packet_fragment[2] > 1486:
#             fragment_missing = True
#             break
#         if index == 0 and packet_fragment[0] != 0:
#             fragment_missing = True
#             break
#         if index > 0 and packet_fragment[0] * b > packet_fragments[index - 1][0] * b + packet_fragments[index - 1][2] - packet_header:
#             fragment_missing = True
#             break
#         if packet_fragment == packet_fragments[-1] and 'MF' == packet_fragment[1]:
#             fragment_missing = True
#             break
#     if fragment_missing:
#         sources.add(packet_id[1])
#         victims.add(packet_id[2])
#         attack_type = "fragmentation attack"
#
# all_traffic = 0
# malicious_traffic = 0
# for packets in pcap_reader('PCAPs_1/hard.pcap', window_size=-1):
#     for packet in packets:
#         proto = None
#         if 'IP' in packet:
#             proto = 'IP'
#         elif 'TCP' in packet:
#             proto = 'TCP'
#         else:
#             proto = 'Ethernet'
#         if packet[proto].dst == list(victims)[0]:
#             all_traffic += 1
#             if packet[proto].src in sources:
#                 malicious_traffic += 1
#
# malicious_ip = 0
# all_ip = 0
# for _, source, destination in packet_dict.keys():
#     if source in sources and destination == list(victims)[0]:
#         malicious_ip += 1
#         all_ip += 1
#     elif destination == list(victims)[0]:
#         all_ip += 1
#
# print(sources)
# print(victims)
# print(f"attack type: {attack_type}")
# print(f"percentage traffic = {malicious_traffic / all_traffic}")
# print(f"percentage ip = {malicious_ip / all_ip}")


{'111.203.126.255', '117.123.91.21', '117.245.186.113', '45.65.205.247', '53.251.154.245', '111.203.36.215', '249.210.217.17', '107.111.123.109', '117.91.75.21', '99.103.57.21', '111.73.155.237', '105.239.71.111', '57.73.153.249', '59.93.135.83', '117.93.59.123', '111.225.120.123', '253.80.251.49', '95.121.219.243', '103.93.103.223', '253.200.183.17', '95.241.123.183', '105.197.109.157', '111.223.202.123', '253.238.167.59', '245.236.159.149', '251.80.89.147', '123.247.91.23', '109.125.15.121', '161.83.57.153', '101.89.139.85', '123.245.101.237', '177.109.21.17', '111.241.43.119', '117.67.247.116', '109.101.61.205', '111.203.11.147', '163.247.216.83', '253.72.43.119', '109.201.184.253', '175.211.74.83', '119.109.167.85', '103.231.56.115', '251.92.255.77', '111.229.139.119', '45.65.59.145', '111.251.9.181', '99.215.106.23', '109.235.142.141', '93.123.107.47', '51.65.197.223', '105.239.71.147', '109.239.46.241', '119.105.173.159', '95.81.183.119', '55.101.251.186', '107.91.95.177', '117.1

### Give an explanation of your method here...

### Task 2.2 Large_2 (2 points)

Analyze the file `PCAPs_2/Large_2`.

In [54]:
# Your code here
class Syn_Flood:
    def __init__(self):
        self.attacker_info = set()
        self.attack_packet_count = 0 #Amount of packets that are considered malicious
        self.received_packets = dict() #Received packets corresponding to the destination address
        self.all_ips = dict() #All IPs that sent to the destination address


    def analyze_packet(self, packet):
        try:
            source_address = packet["IP"].src
            destination_address = packet["IP"].dst

            if destination_address not in self.all_ips:
                self.all_ips[destination_address] = set()
                self.all_ips[destination_address].add(source_address)
            else:
                self.all_ips[destination_address].add(source_address)

            tcp_flag = str(packet["TCP"].flags)

            if destination_address not in self.received_packets:
                self.received_packets[destination_address] = 1
            else:
                self.received_packets[destination_address] = self.received_packets[destination_address] + 1

            if ("S" in tcp_flag and "SA" not in tcp_flag):
                self.attacker_info.add((source_address, tcp_flag, destination_address))
                self.attack_packet_count += 1
            elif("A" == tcp_flag):
                if (source_address, "S", destination_address) in self.attacker_info:
                    self.attack_packet_count -= 1
                    self.attacker_info.remove((source_address, "S", destination_address))
            else:
                pass
        except:
            pass


    def fingerprint(self):
        sources = []
        victim_ip = ""
        count = 0
        for x in self.attacker_info:
            victim_ip = x[2]
            break
        if len(self.attacker_info) > 0:
            print("Attack type: SYN-FLOOD")
            print(f"Victim IP: {victim_ip}")
            for x in self.attacker_info:
                sources.append(x[0])
                count += 1
                if count == 50:
                    break
            print(f"Sources: {sources}")
            print(f"Percentage traffic: {self.attack_packet_count/self.received_packets[victim_ip] * 100}")
            print(f"Percentage IPs: {len(self.attacker_info) / len(self.all_ips[victim_ip]) * 100}")


class DNS_Amp:
    def __init__(self):
        self.attacker_set = set()
        self.victim_set = set()
        self.all_ips = set()
        self.host_dns_query = set()
        self.total_packets = 0
        self.attack_packet_count = 0

    def analyze_packet(self, packet):
        self.all_ips.add(packet["IP"].src)
        self.total_packets += 1
        try:
            source_address = packet["IP"].src
            destination_address = packet["IP"].dst
            source_port = packet["UDP"].sport
            destination_port = packet["UDP"].dport
            if source_port == 53:
                if (destination_address, source_address, destination_port, source_port) not in self.host_dns_query:
                    self.attacker_set.add(source_address)
                    self.victim_set.add(destination_address)
                    self.attack_packet_count += 1
            elif destination_port == 53:
                self.host_dns_query.add((source_address, destination_address, source_port, destination_port))
        except IndexError:
            pass

    def fingerprint(self, attacker_set, victim_set, attack_packet_count, total_packets, all_ips):
        sources = []
        count = 0
        if len(attacker_set) > 0:
            print("Attack type: DNS-AMP")
            for x in victim_set:
                print(f"Victim IP: {x}")
            for x in attacker_set:
                sources.append(x)
                count += 1
                if count == 50:
                    break
            print(f"Sources: {sources}")
            print(f"Percentage traffic: {attack_packet_count/total_packets * 100}")
            print(f"Percentage IPs: {len(attacker_set) / len(all_ips) * 100}")




class DNS_Frag:
    def __init__(self):
        self.all_ips = dict()
        self.received_packets = dict()
        self.attack_packet_count = 0
        self.sources = set()
        self.victims = set()
        self.packet_dict = dict()
        self.packet_counter = 0

    def analyze_packet(self, packet):
        self.packet_counter += 1
        try:
            source_address = packet['IP'].src
            destination_address = packet['IP'].dst
            if destination_address not in self.all_ips:
                self.all_ips[destination_address] = set()
            self.all_ips[destination_address].add(source_address)

            if destination_address not in self.received_packets:
                self.received_packets[destination_address] = 1
            else:
                self.received_packets[destination_address] += 1

            if 'IP' in packet and not 'DNS' in packet:
                if (packet['IP'].id, packet['IP'].src, packet['IP'].dst) not in self.packet_dict:
                    self.packet_dict[(packet['IP'].id, packet['IP'].src, packet['IP'].dst)] = []
                self.packet_dict[(packet['IP'].id, packet['IP'].src, packet['IP'].dst)].append((packet['IP'].frag, packet['IP'].flags, packet['IP'].len, self.packet_counter))
        except IndexError:
            pass


    def fingerprint(self):
        b = 8
        packet_header = 20
        sources = set()
        victims = set()
        malicious_traffic_count = 0
        attack_start_set = False
        attack_start = 0
        attack_end = 0
        for packet_id, packet_fragments in self.packet_dict.items():
            fragment_missing = False
            for index, packet_fragment in enumerate(packet_fragments):
                if packet_fragment[2] > 1486:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
                if index == 0 and packet_fragment[0] != 0:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
                if index > 0 and packet_fragment[0] * b > packet_fragments[index - 1][0] * b + packet_fragments[index - 1][2] - packet_header:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
                if packet_fragment == packet_fragments[-1] and 'MF' == packet_fragment[1]:
                    attack_end = packet_fragment[3]
                    fragment_missing = True
                    if not attack_start_set:
                        attack_start = packet_fragment[3]
                        attack_start_set = True
                    break
            if fragment_missing:
                sources.add(packet_id[1])
                victims.add(packet_id[2])
                malicious_traffic_count += len(packet_fragments)

        if sources:
            print(sources)
            print(victims)
            print(f"percentage traffic = {100 * malicious_traffic_count / self.received_packets[list(victims)[0]]}")
            print(f"percentage ip = {100 * len(sources) / len(self.all_ips[list(victims)[0]])}")
            print(f"attack start = {attack_start}")
            print(f"attack end = {attack_end}")
        else:
            print("There was no Fragmentation attack")

### Give an explanation of your method here...

### Task 2.1 Large_3 (4 points)

The file `PCAPs_2/Large_3` contains an unidentified attack other than the ones you have seen before in this lab. In this task you should take a look at the PCAP and try to identify this attack. After identifying the type of attack, write code to detect the attack and create a fingerprint. 

**In your explanation also explain the attack in addition to how you detected it.**

In [64]:
from math import inf

# Your code here
class ICMP_flood:
    def __init__(self):
        self.all_ips = dict()
        self.received_packets = dict()
        self.attack_packet_count = 0
        self.sources = set()
        self.victims = set()
        self.packet_dict = dict()
        self.packet_counter = 0
        self.first_one = None
        self.last_one = None

    def analyze_packet(self, packet):
        self.packet_counter += 1
        try:
            source_address = packet['IP'].src
            destination_address = packet['IP'].dst
            if destination_address not in self.all_ips:
                self.all_ips[destination_address] = set()
            self.all_ips[destination_address].add(source_address)

            if destination_address not in self.received_packets:
                self.received_packets[destination_address] = 1
            else:
                self.received_packets[destination_address] += 1

            if 'ICMP' in packet and packet['ICMP'].type == 8:
                if (packet['IP'].src, packet['IP'].dst) not in self.packet_dict:
                    self.packet_dict[(packet['IP'].src, packet['IP'].dst)] = []
                self.packet_dict[(packet['IP'].src, packet['IP'].dst)].append(self.packet_counter)

            elif 'ICMP' in packet and packet['ICMP'].type == 0:
                if (packet['IP'].dst, packet['IP'].src) in self.packet_dict:
                    self.packet_dict.pop((packet['IP'].dst, packet['IP'].src))
        except IndexError:
            pass


    def fingerprint(self):
        sources = set()
        victims = set()
        malicious_traffic_count = 0
        first_packet = inf
        last_packet = 0
        for (src, dst), packet_list in self.packet_dict.items():
            sources.add(src)
            victims.add(dst)
            first_packet = min(first_packet, min(packet_list))
            last_packet = max(last_packet, max(packet_list))
            malicious_traffic_count += len(packet_list)

        if sources:
            print(list(sources)[:50])
            print(list(victims)[0])
            print(f"percentage traffic = {100 * malicious_traffic_count / self.received_packets[list(victims)[0]]}")
            print(f"percentage ip = {100 * len(sources) / len(self.all_ips[list(victims)[0]])}")
            print(f"attack start = {first_packet}")
            print(f"attack end = {last_packet}")


icmp_flood = ICMP_flood()
for packets in pcap_reader('PCAPs_2/Large_3.pcap', window_size=50):
        for packet in packets:
            icmp_flood.analyze_packet(packet)

icmp_flood.fingerprint()

['165.193.22.240', '227.214.29.117', '188.86.60.18', '232.171.89.36', '175.174.195.195', '152.18.106.105', '215.151.29.75', '143.1.38.249', '22.27.150.225', '6.149.8.81', '112.78.102.209', '135.175.51.217', '255.1.185.131', '23.165.73.238', '225.246.2.175', '118.44.50.18', '178.200.175.21', '111.190.152.55', '93.97.142.201', '60.93.40.221', '234.210.168.48', '112.38.22.18', '78.58.92.239', '207.83.90.80', '240.169.86.227', '159.49.130.46', '144.193.238.174', '75.10.67.82', '117.250.182.78', '16.217.52.123', '106.7.239.34', '60.118.95.158', '39.202.157.255', '64.48.58.135', '228.198.255.22', '106.183.39.126', '207.96.198.137', '122.2.128.118', '210.48.189.77', '237.179.116.46', '197.118.177.2', '249.70.151.106', '60.105.29.152', '199.229.165.21', '3.169.33.208', '169.64.167.46', '139.142.249.0', '158.158.45.217', '159.254.152.85', '203.19.18.166']
172.27.224.250
percentage traffic = 53.198563638781756
percentage ip = 99.99166736105325
attack start = 2933
attack end = 15636


### Give an explanation of your method here.
After looking at the trace file we observed that there was an unusual
amount of ICMP echo messages concentrated in some places. We researched
the topic and we learnt that there is a type of DDoS attack based
on ICMP pings. Then, we designed algorithm that is supposed to detect
these ICMP pings and provide us with the fingerprint of the attack.
It detects all ICMP messages that are of type 8 (the ping messages).
It also removes the registered ICMP ping from the 'attack'
From the fingerprint we can learn how big the attack was or another
way to think about it - how likely it is that the detected case
was an attack in the first place.